# Data Preprocessing

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, explained_variance_score, r2_score
import tensorflow as tf
import keras_tuner as kt
from sqlalchemy import create_engine

In [2]:
# connect to database
from config import username, pwd
from db_config import endpoint, port, db_name
db_url = f'postgresql://{username}:{pwd}@{endpoint}:{port}/{db_name}'
engine = create_engine(db_url)
conn = engine.connect()

# read in data to pandas
conflict_df = pd.read_sql_table('conflict_dropped', con = conn)
stocks_df = pd.read_sql_table('joined_stocks', con = conn)

# close connection
conn.close()

In [3]:
# convert strings to datetimes
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])
conflict_df['start_date'] = pd.to_datetime(conflict_df['start_date'])
conflict_df['start_date2'] = pd.to_datetime(conflict_df['start_date2'])

In [4]:
# Trying merge
combined_df = conflict_df.merge(stocks_df, how = 'outer', left_on='start_date2', right_on = 'Date')

In [5]:
# gather conflict information that is within the stock availability window.
sorted_stock_dates = stocks_df.Date.sort_values()
available_conflicts = (conflict_df.start_date >= sorted_stock_dates[0]) & (conflict_df.start_date <= sorted_stock_dates.iloc[-1] )

In [6]:
conflict_df[available_conflicts]

,location,side_a,side_a_2nd,side_b,side_b_2nd,incompatibility,territory_name,cumulative_intensity,type_of_conflict,start_date,start_date2,ep_end,region
0,India,Government of India,None,GNLA,None,1,Garoland,0,3,1997-05-29,2012-11-15,1,3
1,India,Government of India,None,GNLA,None,1,Garoland,0,3,1997-05-29,2014-07-01,1,3
6,Sudan,Government of Sudan,None,Republic of South Sudan,None,1,Abyei,0,3,2011-05-01,2011-05-19,1,4
7,South Sudan,Government of South Sudan,None,"SSDM/A, SSLM/A",None,2,None,0,3,2011-08-20,2011-08-20,0,4
8,South Sudan,Government of South Sudan,None,SSLM/A,None,2,None,0,3,2011-08-20,2011-08-20,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2490,Ethiopia,Government of Ethiopia,None,SLM,None,1,Sidamaland,1,3,1981-03-21,1983-03-16,1,4
2491,Myanmar (Burma),Government of Myanmar (Burma),None,MNDAA,None,1,Kokang,0,3,2009-08-27,2009-08-29,1,3
2492,Myanmar (Burma),Government of Myanmar (Burma),None,MNDAA,None,1,Kokang,0,3,2009-08-27,2014-12-22,0,3
2493,Myanmar (Burma),Government of Myanmar (Burma),None,MNDAA,None,1,Kokang,0,3,2009-08-27,2014-12-22,1,3


In [7]:
# create a function to identify the nearest date to get relevant stock rows
def find_nearest_date(date, stock_dates, row_list, not_used_index):
    index = stock_dates.index.get_loc(date, method = 'nearest')
    row_list.append(index)
    if index in not_used_index:
        not_used_index.remove(index)
    return row_list, not_used_index

In [8]:
# create stock_dates_df which has ordered dates as the index for searching
stock_dates_df = stocks_df.set_index('Date').sort_index().dropna()

In [9]:
# create a new DF to append stock data to
available_conflicts_df = conflict_df[available_conflicts].copy()
available_conflicts_df[stock_dates_df.columns] = -999 # default value for easy identification

In [10]:
# create lists for storing stock row indices
stock_rows = []
double_count_index = list(range(stock_dates_df.shape[0])) # used to avoid double counting when merging data later

for index, date in available_conflicts_df.start_date2.iteritems():
    stock_rows, double_count_index = \
    find_nearest_date(date, stock_dates_df, stock_rows, double_count_index)

In [11]:
# insert stock data to available_conflicts_df
available_conflicts_df[stock_dates_df.columns] = stock_dates_df.iloc[stock_rows].values

In [12]:
available_conflicts_df

,location,side_a,side_a_2nd,side_b,side_b_2nd,incompatibility,territory_name,cumulative_intensity,type_of_conflict,start_date,start_date2,ep_end,region,S&P_500,NASDAQ,Dow_Jones
0,India,Government of India,None,GNLA,None,1,Garoland,0,3,1997-05-29,2012-11-15,1,3,1359.88,2853.13,12588.31
1,India,Government of India,None,GNLA,None,1,Garoland,0,3,1997-05-29,2014-07-01,1,3,1985.44,4485.93,17068.26
6,Sudan,Government of Sudan,None,Republic of South Sudan,None,1,Abyei,0,3,2011-05-01,2011-05-19,1,4,1333.27,2803.32,12512.04
7,South Sudan,Government of South Sudan,None,"SSDM/A, SSLM/A",None,2,None,0,3,2011-08-20,2011-08-20,0,4,1123.53,2341.84,10817.65
8,South Sudan,Government of South Sudan,None,SSLM/A,None,2,None,0,3,2011-08-20,2011-08-20,0,4,1123.53,2341.84,10817.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2490,Ethiopia,Government of Ethiopia,None,SLM,None,1,Sidamaland,1,3,1981-03-21,1983-03-16,1,4,149.90,265.01,1117.74
2491,Myanmar (Burma),Government of Myanmar (Burma),None,MNDAA,None,1,Kokang,0,3,2009-08-27,2009-08-29,1,3,1028.93,2028.77,9544.20
2492,Myanmar (Burma),Government of Myanmar (Burma),None,MNDAA,None,1,Kokang,0,3,2009-08-27,2014-12-22,0,3,2070.65,4765.38,17804.80
2493,Myanmar (Burma),Government of Myanmar (Burma),None,MNDAA,None,1,Kokang,0,3,2009-08-27,2014-12-22,1,3,2070.65,4765.38,17804.80


In [13]:
available_conflicts_df[['location',
                        'side_a',
                        'side_b',
                        'territory_name',
                        'start_date',
                        'start_date2',
                        'region',
                        'type_of_conflict',
                        'S&P_500',
                        'NASDAQ',
                        'Dow_Jones']].to_csv('Resources/geo_conflict_data.csv')

In [14]:
# create a function to bin the categorical data
def create_bins(df, col, thresh):
    output = df[col].copy()
    frequencies = output.value_counts()
    categories_to_keep = frequencies[:thresh].index
    output = output.apply(lambda x: x if x in categories_to_keep else 'Other')
    return output

In [15]:
available_conflicts_df.dtypes[available_conflicts_df.dtypes == 'object'].index

Index(['location', 'side_a', 'side_a_2nd', 'side_b', 'side_b_2nd',
       'territory_name', 'region'],
      dtype='object')

In [16]:
my_set = set()
for item in set(available_conflicts_df.side_a_2nd.value_counts().index.to_list()):
    for govt in item.split(', '):
        my_set.add(govt)
len(my_set)
my_set

{'Government of Afghanistan',
 'Government of Angola',
 'Government of Armenia',
 'Government of Australia',
 'Government of Austria',
 'Government of Bangladesh',
 'Government of Belgium',
 'Government of Benin',
 'Government of Bhutan',
 'Government of Bosnia-Herzegovina',
 'Government of Botswana',
 'Government of Burkina Faso',
 'Government of Burundi',
 'Government of Cambodia (Kampuchea)',
 'Government of Cameroon',
 'Government of Canada',
 'Government of Chad',
 'Government of China',
 'Government of Congo',
 'Government of Croatia',
 'Government of Cuba',
 'Government of Czech Republic',
 'Government of DR Congo (Zaire)',
 'Government of Denmark',
 'Government of Djibouti',
 'Government of Egypt',
 'Government of El Salvador',
 'Government of Estonia',
 'Government of Ethiopia',
 'Government of Finland',
 'Government of France',
 'Government of Gambia',
 'Government of Germany',
 'Government of Ghana',
 'Government of Greece',
 'Government of Guatemala',
 'Government of Guinea

In [17]:
# get string columns for encoding
string_cols = available_conflicts_df.dtypes[available_conflicts_df.dtypes == 'object'].index
ignore_cols = ['side_a_2nd', 'side_b_2nd', 'region'] # can handle later if necessary
string_cols = string_cols.drop(labels = ignore_cols)

# bin the categorical data 
names_to_keep = 25 # number of bins = names_to_keep + 1
for col in string_cols:
    available_conflicts_df[col] = create_bins(available_conflicts_df, col, names_to_keep)

# hand encode region columns 1,2,3,4,5 before encoding the rest
available_conflicts_df['region_1'] = 0
available_conflicts_df['region_2'] = 0
available_conflicts_df['region_3'] = 0
available_conflicts_df['region_4'] = 0
available_conflicts_df['region_5'] = 0

# store the conflict region strings as a series
conflict_regions = available_conflicts_df.region.copy()

# loop over the conflict_region strings
for row, regions_string in enumerate(conflict_regions):
    
    # split the strings
    regions = regions_string.split(',')
    # extract the row
    row_data = available_conflicts_df.iloc[row,-5:].copy()
    for region in regions:
        row_data['region_' + str(int(region))] = 1
    available_conflicts_df.iloc[row, -5:] = row_data
available_conflicts_df
# encode the data
binned_df = pd.get_dummies(available_conflicts_df, columns = string_cols)

# add a Date column to binned_df for merging
binned_df['Date'] = binned_df.start_date2

In [18]:
# merge on date and stock data
combined_df = binned_df.merge(
    stock_dates_df.iloc[double_count_index],
    on=['Date', 'S&P_500','NASDAQ','Dow_Jones'],
    how = 'outer'
)

# create year, month, day columns
combined_df['year'] = combined_df.Date.apply(lambda date: date.year)
combined_df['month'] = combined_df.Date.apply(lambda date: date.month)
combined_df['day'] = combined_df.Date.apply(lambda date: date.day)

In [19]:
# drop rows that have missing stock data
combined_df = combined_df[combined_df['S&P_500'].notna()]
combined_df = combined_df[combined_df['NASDAQ'].notna()]
combined_df = combined_df[combined_df['Dow_Jones'].notna()]

In [20]:
# drop start_date and start_date2
combined_df = combined_df.drop(columns = ['start_date', 'start_date2'])

# fill in NaNs
combined_df = combined_df.fillna(0)

In [21]:
# output csv file
output_path = os.path.join('Resources', 'combined_data_1.csv')
combined_df.to_csv(output_path, index = False)

# Starting ML

In [22]:
# drop unnecessary columns from data
ignore_cols.append('Date')
data = combined_df.drop(columns = ignore_cols).drop_duplicates()
data

,incompatibility,cumulative_intensity,type_of_conflict,ep_end,S&P_500,NASDAQ,Dow_Jones,region_1,region_2,region_3,...,territory_name_Novorossiya,territory_name_Other,territory_name_Punjab/Khalistan,territory_name_Rojava Kurdistan,territory_name_Serb,territory_name_Southern Lebanon,territory_name_Tripura,year,month,day
0,1.0,0.0,3.0,1.0,1359.88,2853.13,12588.31,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2012,11,15
1,1.0,0.0,3.0,1.0,1985.44,4485.93,17068.26,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2014,7,1
2,1.0,0.0,3.0,1.0,1333.27,2803.32,12512.04,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2011,5,19
3,2.0,0.0,3.0,0.0,1123.53,2341.84,10817.65,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2011,8,20
5,2.0,1.0,3.0,0.0,1123.53,2341.84,10817.65,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2011,8,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2592,0.0,0.0,0.0,0.0,2423.41,6140.42,21349.63,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017,6,30
2593,0.0,0.0,0.0,0.0,2472.54,6387.75,21580.07,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017,7,21
2594,0.0,0.0,0.0,0.0,2472.10,6374.68,21830.31,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017,7,28
2595,0.0,0.0,0.0,0.0,2441.32,6256.56,21858.32,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017,8,11


In [23]:
# extract X and y from data
y_cols = ['S&P_500', 'NASDAQ', 'Dow_Jones']
X = data.drop(columns = y_cols)
# converts all column dtypes to string instead of some being sqlalchemy quoted_name
X.columns = X.columns.map(str)
y = data[y_cols]

In [24]:
# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, train_size=.8)

In [25]:
# scale data
X_scaler = StandardScaler()

X_train_scaled = X_scaler.fit_transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Extra Random Trees Regressor

In [26]:
# create regressor
# n_jobs = -1 means use all the processors on your PC to compute the decision trees in parallel
erf = ExtraTreesRegressor(n_estimators = 100, random_state = 42, n_jobs = -1)

# fit regressor/train model
erf.fit(X_train_scaled, y_train)

# make predictions
y_pred = erf.predict(X_test_scaled)

# covert predictions into dataframe
y_pred_df = pd.DataFrame(y_pred.round(2), index = y_test.index, columns = y_test.columns)

In [27]:
mean_squared_error(y_test, y_pred_df, squared=False, multioutput='raw_values')

array([ 22.02850271,  68.77912573, 189.16333963])

In [28]:
r2_score(y_test, y_pred_df)

0.9984806609299305

In [29]:
(((np.abs(y_test-y_pred_df) / y_test) * 100)).max()

S&P_500      22.906249
NASDAQ       35.344431
Dow_Jones    19.303284
dtype: float64

In [30]:
for importance, feature in sorted(zip(erf.feature_importances_, X.columns), reverse=True):
    print(feature, importance)

year 0.9756062910119059
territory_name_Islamic State 0.009155322168726167
month 0.006580653286561413
side_b_IS 0.0063633516866222085
day 0.000586633244976317
type_of_conflict 0.00031497476012634575
cumulative_intensity 0.00012355802711623
incompatibility 0.00011062994633272095
region_2 9.597059992079662e-05
region_4 7.989392371598975e-05
location_Pakistan 7.984265410978841e-05
side_a_Government of Pakistan 7.039532616095372e-05
side_b_FDLR 5.387952451175048e-05
side_b_Other 4.3877885452193724e-05
side_a_Government of Rwanda 4.138589595800367e-05
side_a_Government of South Sudan 3.971659872509006e-05
location_Syria 3.8835011501314166e-05
territory_name_Other 3.81461671288264e-05
location_Egypt 3.6874184826611745e-05
side_a_Government of Syria 3.6602136008178745e-05
region_1 3.2984996143508184e-05
region_3 3.187820292944761e-05
location_South Sudan 2.940457416019084e-05
side_a_Government of Russia (Soviet Union) 2.7714512716430746e-05
ep_end 2.6509492076010056e-05
location_Rwanda 2.54087

## Neural Network

In [31]:
# TODO: Keras Tuner Neural Network